In [1]:
import pickle
import os
import sys
import pandas as pd
import numpy as np
from numpy import mean as npMean
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import seaborn as sns
import umap
import re

import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model ,load_model


from lifelines import CoxPHFitter
from Models.RCFR_noRiskRank import SetModel
from Module.DataProcessing import DataLoad
from Module.MetricsGroup import DoMetric, DoAggMetric, DoSimEval

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="3"





### Data load

In [2]:
StackedData, IntToGene, TTE, EVENT, TrIndEmbeddMask, ReferencePatIDLong, ReferencePatIDShort, NormDismInd, MergedData= DataLoad()

PatIDX = StackedData[:, 0:1].astype('int')
GeneIDX = StackedData[:, 1:2].astype('int')
GeneExp = StackedData[:, 2:3]

IndN = len(np.unique(PatIDX))
FeatN = len(np.unique(GeneIDX))

## Setting for tasks

In [3]:
ModelList = os.listdir('./Results/')
ModelList = [i for i in ModelList if 'M02' in i and 'NoRiskRank' in i ]
FilePath = './Results/'

# Parameters for post-hoc models
EmbedSize = 50
NCL_Feat = 5
NCL_Ind = 2
AdjCosWeight_ = 1. # This parameter affects only training phases, so thus any float number can be set in the post analysis phases. 
NumGene_CL = 100

# Model structure load
RCFR, LayerList = SetModel( NormDismInd, TrIndEmbeddMask, IndN, FeatN, ReferencePatIDLong, ReferencePatIDShort)

# Data for calculating metric
DataMetric = [MergedData, TTE, EVENT, NCL_Ind, NCL_Feat, NumGene_CL, IntToGene]


ColList = ['Model','AvgtPRate', 'AvgtAdjPRate', 'MintAdjPRate', 'AvgABSGeCohD', 'MinABSGeCohD', 'AvgABSSurvCoef', 'MinABSSurvCoef', 'AvgSurvpVal', 
           'MaxSurvpVal', 'NegExpAvgSurvpVal', 'NegExpMinSurvpVal', 'AvgNegSigRate',  'MinNegSigRate', 'AvgPosSigRate', 'MinPosSigRate','IndCentRatio']


## Procedure for Selecting best mode

In [42]:
MetricTable = pd.DataFrame(columns=ColList)
InfoFeatGroupList = []

for num, model in enumerate(ModelList[:]):
    print(num)
    

    RCFR.load_weights(FilePath + model)  # Model weights load
    InpInd, InpFeat, IndEmbeddWeig, IndEmbeddReferenceLong, FeatEmbeddWeig, IndCentroid, FeatCentroid, ICosCLSim, FCosCLSim = LayerList
        
    # Metric calculation: InfoFeatGroup will be used in UMAP analysis
    metrics, InfoFeatGroup = DoMetric (DataMetric, [InpInd, InpFeat, IndEmbeddWeig, FeatEmbeddWeig, IndCentroid, FeatCentroid, ICosCLSim, FCosCLSim])
    InfoFeatGroupList.append(InfoFeatGroup)
    print(InfoFeatGroup)
    MetricTable = pd.concat([MetricTable, pd.DataFrame([[model] + metrics], columns=ColList)], axis=0)


#MetricTable = DoAggMetric(AggMetricList, MetricTable)
MetricTable['GroupM'] = np.array([re.findall('.\d+', i)[1][1:] for i in  MetricTable['Model']])
MetricTable['EpNum'] = np.array([ re.findall('.\d+\.', i)[0][1:-1] for i in  MetricTable['Model']]).astype('int')
MetricTable = MetricTable.sort_values(['GroupM','EpNum'])

MetricTable.to_csv('./MetricTables/RCFR_NoRiskRank_MetricTable.csv',index=False)

0
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
[array([0., 0., 0., 0.]), array([0., 0., 0., 0.]), [0.612, 0.333, 0.544, 0.621]]
1
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
[array([0.   , 0.   , 0.   , 0.014]), array([0.   , 0.   , 0.   , 0.004]), [0.899, 0.061, 0.778, 0.459]]
2
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
[array([0.118, 0.89 , 0.102, 0.164]), array([0.   , 0.   , 0.026, 0.128]), [0.282, 0.011, 0.15, 0.003]]
3
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
[array([0.052, 0.778, 0.   , 0.2  ]), array([0.   , 0.002, 0.   , 0.18 ]), [0.003, 0.0, 0.0, 0.0]]
4
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
[array([0.03 , 0.788, 0.   , 0.152]), array([0.   , 0.006, 0.   , 0.278]), [0.001, 0.0, 0.0, 0.0]]
5
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
[array([0.018, 0.798, 0.044, 0.176]), array([0.   , 0.004, 0.002, 0.

c:\users\user\anaconda3\envs\ai\lib\site-packages\lifelines\fitters\coxph_fitter.py:1594: ConvergenceWarning: Newton-Rhaphson convergence completed successfully but norm(delta) is still high, 0.497. This may imply non-unique solutions to the maximum likelihood. Perhaps there is collinearity or complete separation in the dataset?

  warnings.warn(


0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
[array([0.076, 0.872, 0.076, 0.212]), array([0.002, 0.002, 0.006, 0.162]), [0.994, 0.0, 0.0, 0.0]]
14
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
[array([0.126, 0.894, 0.074, 0.202]), array([0.002, 0.004, 0.002, 0.148]), [0.0, 0.0, 0.0, 0.0]]
15
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
[array([0.134, 0.866, 0.092, 0.172]), array([0.002, 0.002, 0.002, 0.172]), [0.0, 0.0, 0.0, 0.0]]
16
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
[array([0.1  , 0.858, 0.028, 0.15 ]), array([0.002, 0.   , 0.   , 0.122]), [0.0, 0.0, 0.0, 0.0]]
17
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
[array([0.088, 0.866, 0.008, 0.17 ]), array([0.002, 0.   , 0.   , 0.132]), [0.0, 0.0, 0.0, 0.0]]
18
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
[array([0.134, 0.826, 0.   , 0.18 ]), array([0.002, 0.   , 0.   , 

c:\users\user\anaconda3\envs\ai\lib\site-packages\lifelines\fitters\coxph_fitter.py:1594: ConvergenceWarning: Newton-Rhaphson convergence completed successfully but norm(delta) is still high, 0.495. This may imply non-unique solutions to the maximum likelihood. Perhaps there is collinearity or complete separation in the dataset?

  warnings.warn(


0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
[array([0.17 , 0.962, 0.054, 0.136]), array([0.002, 0.   , 0.   , 0.158]), [0.993, 0.0, 0.0, 0.0]]
30
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
[array([0.202, 0.976, 0.   , 0.11 ]), array([0.002, 0.   , 0.   , 0.17 ]), [0.0, 0.0, 0.0, 0.0]]
31
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
[array([0.104, 0.966, 0.028, 0.112]), array([0.002, 0.   , 0.   , 0.168]), [0.0, 0.0, 0.0, 0.0]]
32
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
[array([0.126, 0.96 , 0.002, 0.114]), array([0.   , 0.   , 0.   , 0.152]), [0.0, 0.0, 0.0, 0.0]]
33
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
[array([0.252, 0.944, 0.002, 0.108]), array([0.   , 0.   , 0.   , 0.176]), [0.0, 0.0, 0.0, 0.0]]
34
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
[array([0.214, 0.942, 0.   , 0.104]), array([0.   , 0.   , 0.   , 

...

0
100
200
300
400
0
100
200
300
400


c:\users\user\anaconda3\envs\ai\lib\site-packages\lifelines\fitters\coxph_fitter.py:1666: RuntimeWarning: overflow encountered in exp
  scores = weights * exp(dot(X, beta))
c:\users\user\anaconda3\envs\ai\lib\site-packages\lifelines\fitters\coxph_fitter.py:1718: RuntimeWarning: divide by zero encountered in true_divide
  denom = 1.0 / np.array([risk_phi])
c:\users\user\anaconda3\envs\ai\lib\site-packages\lifelines\fitters\coxph_fitter.py:1720: RuntimeWarning: invalid value encountered in multiply
  a1 = risk_phi_x_x * denom
c:\users\user\anaconda3\envs\ai\lib\site-packages\lifelines\fitters\coxph_fitter.py:1722: RuntimeWarning: invalid value encountered in multiply
  summand = numer * denom[:, None]
c:\users\user\anaconda3\envs\ai\lib\site-packages\lifelines\fitters\coxph_fitter.py:1714: RuntimeWarning: divide by zero encountered in true_divide
  denom = 1.0 / (risk_phi - increasing_proportion * tie_phi)
c:\users\user\anaconda3\envs\ai\lib\site-packages\lifelines\fitters\coxph_fitter.p

0
100
200
300
400
0
100
200
300
400
[array([0.   , 0.926, 0.   , 0.188]), array([0.08, 0.  , 0.  , 0.04]), [0.996, 0.0, 1.0, 0.0]]
496


c:\users\user\anaconda3\envs\ai\lib\site-packages\lifelines\utils\__init__.py:1123: ConvergenceWarning: Column IndCentMembers have very low variance when conditioned on death event present or not. This may harm convergence. This could be a form of 'complete separation'. For example, try the following code:

>>> events = df['event'].astype(bool)
>>> print(df.loc[events, 'IndCentMembers'].var())
>>> print(df.loc[~events, 'IndCentMembers'].var())

A very low variance means that the column IndCentMembers completely determines whether a subject dies or not. See https://stats.stackexchange.com/questions/11109/how-to-deal-with-perfect-separation-in-logistic-regression.

  warnings.warn(dedent(warning_text), ConvergenceWarning)
c:\users\user\anaconda3\envs\ai\lib\site-packages\lifelines\fitters\coxph_fitter.py:1594: ConvergenceWarning: Newton-Rhaphson convergence completed successfully but norm(delta) is still high, 0.398. This may imply non-unique solutions to the maximum likelihood. Perhaps 

0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
[array([0.   , 0.922, 0.   , 0.118]), array([0.092, 0.   , 0.   , 0.014]), [0.995, 0.0, 1.0, 0.0]]
497
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
[array([0.854, 0.998, 0.658, 0.862]), array([0., 0., 0., 0.]), [0.0, 0.0, 0.0, 0.0]]
498
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
[array([0.844, 0.996, 0.5  , 0.468]), array([0.   , 0.   , 0.   , 0.032]), [0.0, 0.0, 0.0, 0.0]]
499
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
[array([0.8  , 0.988, 0.252, 0.434]), array([0.   , 0.002, 0.   , 0.042]), [0.0, 0.0, 0.0, 0.0]]


## Priority-based filter out by metrics

In [4]:
def Aggregation(MetricTable,AggMetricList):
    AggMetricTable = DoSimEval(MetricTable, 'MaxSurvpVal',pCutoff, AggMetricList, ExcRate, NmodEahG)
    AggMetricRank = DoAggMetric(AggMetricList, AggMetricTable[['Model']+AggMetricList]).sort_values('Metrics')
    AggMetricRank = pd.merge(AggMetricRank, AggMetricTable[['Model','MaxSurvpVal']], on='Model', how='left')
    BestModel = AggMetricRank.sort_values('Metrics').iloc[-1]
    
    return AggMetricRank, BestModel

In [5]:
pCutoff = 0.005
ExcRate = 0.2
NmodEahG = 1

NegMetricList = ['IndCentRatio', 'MinABSSurvCoef', 'AvgABSSurvCoef',  'MinNegSigRate', 'AvgNegSigRate', 'MinABSGeCohD', 'AvgABSGeCohD']
PosMetricList = ['IndCentRatio', 'MinABSSurvCoef', 'AvgABSSurvCoef', 'MinPosSigRate', 'AvgPosSigRate', 'MinABSGeCohD', 'AvgABSGeCohD']

MetricTable = pd.read_csv('./MetricTables/RCFR_NoRiskRank_MetricTable.csv')
MetricTable['GroupM'] = np.array([re.findall('.\d+', i)[1][1:] for i in  MetricTable['Model']])
MetricTable['EpNum'] = np.array([ re.findall('.\d+\.', i)[0][1:-1] for i in  MetricTable['Model']]).astype('int')


In [6]:
NegAggMetricRank, NegBestModel =  Aggregation(MetricTable, NegMetricList)
PosAggMetricRank, PosBestModel =  Aggregation(MetricTable, PosMetricList)

NegAggMetricRank.to_csv('./MetricTables/RCFR_NoRisk_Neg_AggMetricRank_Filt'+str(NumGene_CL)+'.csv',index=False)
PosAggMetricRank.to_csv('./MetricTables/RCFR_NoRisk_Pos_AggMetricRank_Filt'+str(NumGene_CL)+'.csv',index=False)

1
N obs with filter of IndCentRatio : 69
N obs with filter of MinABSSurvCoef : 56
N obs with filter of AvgABSSurvCoef : 45
N obs with filter of MinNegSigRate : 36
N obs with filter of AvgNegSigRate : 29
N obs with filter of MinABSGeCohD : 24
N obs with filter of AvgABSGeCohD : 20

2
N obs with filter of IndCentRatio : 71
N obs with filter of MinABSSurvCoef : 57
N obs with filter of AvgABSSurvCoef : 46
N obs with filter of MinNegSigRate : 37
N obs with filter of AvgNegSigRate : 30
N obs with filter of MinABSGeCohD : 24
N obs with filter of AvgABSGeCohD : 20

3
N obs with filter of IndCentRatio : 63
N obs with filter of MinABSSurvCoef : 51
N obs with filter of AvgABSSurvCoef : 41
N obs with filter of MinNegSigRate : 33
N obs with filter of AvgNegSigRate : 27
N obs with filter of MinABSGeCohD : 22
N obs with filter of AvgABSGeCohD : 18

4
N obs with filter of IndCentRatio : 56
N obs with filter of MinABSSurvCoef : 45
N obs with filter of AvgABSSurvCoef : 36
N obs with filter of MinNegSigR

In [58]:
NegAggMetricRank[NegMetricList]

,IndCentRatio,MinABSSurvCoef,AvgABSSurvCoef,MinNegSigRate,AvgNegSigRate,MinABSGeCohD,AvgABSGeCohD
0,0.523810,3.803062,4.186322,0.046,0.3525,0.108205,0.208659
1,0.422222,4.075867,4.288296,0.102,0.4090,0.120220,0.209306
2,0.427101,4.045290,4.210118,0.102,0.4145,0.120582,0.216268
3,0.432014,3.514848,3.964598,0.128,0.4850,0.128522,0.227590
4,0.439446,3.869502,4.019744,0.124,0.4660,0.124597,0.224371


In [59]:
PosAggMetricRank[PosMetricList]

,IndCentRatio,MinABSSurvCoef,AvgABSSurvCoef,MinPosSigRate,AvgPosSigRate,MinABSGeCohD,AvgABSGeCohD
0,0.507246,4.064988,4.701705,0.002,0.039,0.090825,0.183350
1,0.507246,4.064988,4.704152,0.002,0.039,0.092246,0.182636
2,0.507246,4.064988,4.701705,0.002,0.040,0.090873,0.182510
3,0.529412,3.974790,4.365373,0.000,0.041,0.109130,0.215198
4,0.507246,4.064988,4.704152,0.002,0.041,0.091612,0.182862
